In [1]:
import os
import cv2
from PIL import Image
from ultralytics import YOLO

# Configurazione
SIZE = (640, 360)
MODEL_PATH = "/home/lorenzo/Scaricati/license_plate_detector.pt"
INPUT_DIR = "/home/lorenzo/Scrivania/tmp"
OUTPUT_DIR = os.path.join(INPUT_DIR, "converted_jpg")

# Carica il modello YOLO per il riconoscimento delle targhe
model = YOLO(MODEL_PATH)

os.makedirs(OUTPUT_DIR, exist_ok=True)

def process_images(directory):
    for filename in os.listdir(directory):
        if filename.lower().endswith(".jpg"):
            img_path = os.path.join(directory, filename)
            output_path = os.path.join(OUTPUT_DIR, filename)

            # Apri l'immagine con OpenCV
            image = cv2.imread(img_path)
            if image is None:
                print(f"Errore nel caricamento: {img_path}")
                continue

            # Rilevamento delle targhe con YOLO
            results = model(image)
            for result in results:
                for box in result.boxes.xyxy:
                    x1, y1, x2, y2 = map(int, box)
                    plate_roi = image[y1:y2, x1:x2]
                    blurred_roi = cv2.GaussianBlur(plate_roi, (101, 101), 0)
                    image[y1:y2, x1:x2] = blurred_roi

            # Converti l'immagine in formato PIL per ridimensionarla
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(image)
            pil_img = pil_img.resize(SIZE)
            
            # Salva l'immagine elaborata
            pil_img.save(output_path, "JPEG", quality=95)
            print(f"Processato: {img_path} -> {output_path}")

if __name__ == "__main__":
    process_images(INPUT_DIR)
    print("Elaborazione completata!")


0: 384x640 (no detections), 103.2ms
Speed: 6.9ms preprocess, 103.2ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)
Processato: /home/lorenzo/Scrivania/tmp/20250223_144944.jpg -> /home/lorenzo/Scrivania/tmp/converted_jpg/20250223_144944.jpg

0: 384x640 (no detections), 89.7ms
Speed: 2.2ms preprocess, 89.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Processato: /home/lorenzo/Scrivania/tmp/20250223_143954.jpg -> /home/lorenzo/Scrivania/tmp/converted_jpg/20250223_143954.jpg

0: 384x640 (no detections), 62.2ms
Speed: 3.2ms preprocess, 62.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Processato: /home/lorenzo/Scrivania/tmp/20250223_143958.jpg -> /home/lorenzo/Scrivania/tmp/converted_jpg/20250223_143958.jpg

0: 384x640 (no detections), 66.3ms
Speed: 2.3ms preprocess, 66.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Processato: /home/lorenzo/Scrivania/tmp/20250223_143847.jpg -> /home/lorenzo/Scrivania/tmp/

In [1]:
import os
import shutil
import random

def split_dataset(base_path, valid_ratio=0.05):
    # Definisce i path per le cartelle train e valid
    train_images = os.path.join(base_path, "train", "images")
    train_labels = os.path.join(base_path, "train", "labels")
    valid_images = os.path.join(base_path, "valid", "images")
    valid_labels = os.path.join(base_path, "valid", "labels")

    # Crea le directory di valid se non esistono
    os.makedirs(valid_images, exist_ok=True)
    os.makedirs(valid_labels, exist_ok=True)

    # Elenca tutti i file immagini in train/images
    image_files = [f for f in os.listdir(train_images) if os.path.isfile(os.path.join(train_images, f))]
    total_images = len(image_files)
    
    # Calcola quanti file spostare (almeno 1)
    num_to_move = max(1, int(total_images * valid_ratio))
    print(f"Totale immagini in train: {total_images}. Verranno spostate {num_to_move} immagini (e relativi label) in valid.")

    # Seleziona casualmente i file da spostare
    files_to_move = random.sample(image_files, num_to_move)

    for filename in files_to_move:
        # Sposta l'immagine dalla cartella train/images a valid/images
        src_img = os.path.join(train_images, filename)
        dst_img = os.path.join(valid_images, filename)
        shutil.move(src_img, dst_img)
        print(f"Spostata immagine: {src_img} -> {dst_img}")

        # Sposta il file label corrispondente se esiste
        src_label = os.path.join(train_labels, filename[:-3] + "txt")
        dst_label = os.path.join(valid_labels, filename[:-3] + "txt")
        if os.path.exists(src_label):
            shutil.move(src_label, dst_label)
            print(f"Spostato label: {src_label} -> {dst_label}")
        else:
            print(f"Nessun label trovato per l'immagine: {filename}")

# Imposta qui il percorso della cartella che contiene "train" e "valid"
path = "/home/lorenzo/Scrivania/new_images/converted_jpg/data"  # Modifica questo percorso

# Esegui lo split del dataset
split_dataset(path)

Totale immagini in train: 1446. Verranno spostate 72 immagini (e relativi label) in valid.
Spostata immagine: /home/lorenzo/Scrivania/new_images/converted_jpg/data/train/images/vlcsnap-2025-02-19-17h02m57s019.jpg -> /home/lorenzo/Scrivania/new_images/converted_jpg/data/valid/images/vlcsnap-2025-02-19-17h02m57s019.jpg
Spostato label: /home/lorenzo/Scrivania/new_images/converted_jpg/data/train/labels/vlcsnap-2025-02-19-17h02m57s019.txt -> /home/lorenzo/Scrivania/new_images/converted_jpg/data/valid/labels/vlcsnap-2025-02-19-17h02m57s019.txt
Spostata immagine: /home/lorenzo/Scrivania/new_images/converted_jpg/data/train/images/vlcsnap-2025-02-19-14h23m51s336.jpg -> /home/lorenzo/Scrivania/new_images/converted_jpg/data/valid/images/vlcsnap-2025-02-19-14h23m51s336.jpg
Spostato label: /home/lorenzo/Scrivania/new_images/converted_jpg/data/train/labels/vlcsnap-2025-02-19-14h23m51s336.txt -> /home/lorenzo/Scrivania/new_images/converted_jpg/data/valid/labels/vlcsnap-2025-02-19-14h23m51s336.txt
Spo